In [4]:
%matplotlib inline
import pandas as pd
import numpy as np
from copy import deepcopy
from matplotlib import pyplot as plt

import os
import datetime
import time
import json
import pickle
import os

In [6]:
with open("config.json", "r") as f:
    config = json.load(f)
folder_path = config["data_root_path"]
focus_path = folder_path + "Selection_Group/" + config["label_focus"]
epis_long_to_short = config["epis_long_to_short"]
epis = config["epis"]
wks = config["wks"]

In [ ]:
if (not os.path.exists(focus_path)):
    os.mkdir(focus_path)

In [7]:
df_phaseI = pd.read_csv(folder_path + "aggregated_all_phaseI.csv")

C:\ProgramPool\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (45,55,58,61,64,67,69,70,71,72,73,74,75,76,82,83,91,173,178,183,188,193,198,203,430,570,572,573,1490,1491,1492,1493,1494,1543,2191,2192,2196,2197,2201,2202,2206,2207,2211,2212,2216,2217,2221,2222,2226,2227,2231,2232,2236,2237,2241,2242,2246,2247,2251,2252,2256,2257,2261,2262,2266,2267,2271,2272,2276,2277,2281,2282,2286,2287,2291,2292,2296,2297,2301,2302,2306,2307,2311,2312,2316,2317,2321,2322,2326,2327,2331,2332) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df_phaseII = pd.read_csv(folder_path + "aggregated_all_phaseII.csv")

C:\ProgramPool\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (59,61,63,65,99,100,101,102,103,104,105,106,107,108,111,112,192,195,197,200,202,205,207,210,212,215,217,220,222,225,580,592,594,595,1565,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815,1816,1817,1818,1819,1820,1821,1822,1823,1824,1825,1826,1827,1828,1829,1830,1831,1832,1833,1834,1835,1836,1837,1838,1839,1840,1841,1842,1843,1844,1845,1846,1847,1848,1849,1850,1851,1852,1853,1854,1855,1856,1857,1858,1859,1860,1861,1862,1863,1864,1865,1866,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081,2082,2083,2084,2085,2086,2087,2088,

In [85]:
pid_counts.shape

(228, 6)

In [84]:
df_phaseI_grp = df_phaseII.groupby("PID")
cols = [
"f_screen_number_samples_screen_allday",
"f_call_number_rows_calls_allday",
"f_blue_number_samples_bluetooth_allday",
"f_loc_number_samples_location_allday",
"f_slp_num_samples_sleep_allday",
"f_steps_num_samples_steps_allday"]
pid_counts = []
for i, df_buf in df_phaseI_grp:
    if (sum(df_buf["BDI_II_POST"] >= 0) > 0):
        pid_counts.append(df_buf[cols].sum().tolist())
pid_counts = pd.DataFrame(pid_counts)
for i,j in zip(pid_counts.mean(axis = 0), pid_counts.std(axis = 0)):
    print(round(i,1), round(j,1))

9103.2 13695.3
92.7 98.5
48914.0 137602.5
4932.9 2917.5
2099.8 8488.8
121414.7 64491.3


In [7]:
phase_I2II_mapping = pd.read_csv(folder_path + "PID-mapping.csv")

In [10]:
phase_I2II_mapping = phase_I2II_mapping.reset_index()
phase_I2II_mapping.rename({"index":"PID_new"}, axis = 1, inplace = True)
phase_I2II_mapping["PID_new"] += 1

In [17]:
overlap_phaseI = list(set(phase_I2II_mapping["PhaseI"].tolist()))
overlap_phaseII = list(set(phase_I2II_mapping["PhaseII"].tolist()))

In [182]:
df_phaseI_select = deepcopy(df_phaseI[df_phaseI["PID"].apply(lambda x : x in overlap_phaseI)])

In [169]:
df_phaseI_select.shape

(7950, 2455)

In [183]:
df_phaseII_select = deepcopy(df_phaseII[df_phaseII["PID"].apply(lambda x : x in overlap_phaseII)])

In [171]:
df_phaseII_select.shape

(8856, 2267)

In [184]:
df_phaseI_select["PID"] = df_phaseI_select["PID"].map({r["PhaseI"]:r["PID_new"] for i, r in phase_I2II_mapping.iterrows()})
df_phaseII_select["PID"] = df_phaseII_select["PID"].map({r["PhaseII"]:r["PID_new"] for i, r in phase_I2II_mapping.iterrows()})
df_phaseI_select["PID"] = df_phaseI_select["PID"].astype(str)
df_phaseII_select["PID"] = df_phaseII_select["PID"].astype(str)

In [185]:
df_phaseI_select["phase"] = "I"
df_phaseII_select["phase"] = "II"
df_phaseI_select["wk"] = df_phaseI_select["date"].apply(lambda x : "wkdy" if pd.to_datetime(x).weekday() + 1 < 6 else "wkend")
df_phaseII_select["wk"] = df_phaseII_select["date"].apply(lambda x : "wkdy" if pd.to_datetime(x).weekday() + 1 < 6 else "wkend")

In [186]:
overlap_cols_ = list(set(df_phaseI_select.columns.tolist()).intersection(set(df_phaseII_select.columns.tolist())))
overlap_cols = [x for x in df_phaseI_select.columns.tolist() if x in overlap_cols_]
overlap_feature_cols = overlap_cols[:3] + overlap_cols[99:]

In [187]:
df_phaseI_select = deepcopy(df_phaseI_select[overlap_feature_cols])
df_phaseII_select = deepcopy(df_phaseII_select[overlap_feature_cols])

In [188]:
cols = df_phaseII_select.columns.tolist()
cols = cols[:3] + cols[-2:] + cols[3:-2]
df_phaseI_select = df_phaseI_select[cols]
df_phaseII_select = df_phaseII_select[cols]

In [51]:
def from_two_cols_to_dict(x):
    return {r[0]:r[1] for i, r in x.iterrows()}

In [102]:
phaseI_BDI = pd.DataFrame([[k,v] for k,v in from_two_cols_to_dict(df_phaseI_select[["PID","BDI_II_POST"]]).items()],columns = ["PID","BDI_I"])
phaseII_BDI = pd.DataFrame([[k,v] for k,v in from_two_cols_to_dict(df_phaseII_select[["PID","BDI_II_POST"]]).items()],columns = ["PID","BDI_II"])

In [103]:
df_BDI = phaseI_BDI.merge(phaseII_BDI, on = "PID", how = "left")

In [104]:
df_BDI = df_BDI[(df_BDI["BDI_I"] != 999) &(~df_BDI["BDI_II"].isna())]

In [191]:
df_BDI = df_BDI.sort_values("PID")
df_BDI["label_I"] = df_BDI["BDI_I"] < 14
df_BDI["label_II"] = df_BDI["BDI_II"] < 14

In [110]:
sum(df_BDI["BDI_I"] >= 14)

24

In [111]:
sum(df_BDI["BDI_II"] >= 14)

27

In [112]:
sum((df_BDI["BDI_I"] >= 14) & (df_BDI["BDI_II"] >= 14))

17

In [193]:
pool = {"data_phaseI":df_phaseI_select,
       "data_phaseII":df_phaseII_select,
       "labels":df_BDI}

In [194]:
with open(focus_path + "raw_data_pool.pkl", "wb") as f:
    pickle.dump(pool,f)

In [204]:
feature_col_all = df_phaseI_select.columns.tolist()[5:]
feature_col_eps = {}
for f in feature_col_all:
    ep_long = f.split("_")[-1]
    if ("allday" in ep_long or "_dis_" in f):
        continue
    else:
        if (eps_long_to_short[ep_long] not in feature_col_eps):
            feature_col_eps[eps_long_to_short[ep_long]] = [f]
        else:
            feature_col_eps[eps_long_to_short[ep_long]].append(f)
            
feature_dis_col_eps = {}
for f in feature_col_all:
    ep_long = f.split("_")[-1]
    if ("allday" in ep_long or "_dis_" not in f):
        continue
    else:
        if (eps_long_to_short[ep_long] not in feature_dis_col_eps):
            feature_dis_col_eps[eps_long_to_short[ep_long]] = [f]
        else:
            feature_dis_col_eps[eps_long_to_short[ep_long]].append(f)
with open(folder_path + "feature_col_eps.json","w") as f:
    json.dump(feature_col_eps,f)
with open(folder_path + "feature_dis_col_eps.json","w") as f:
    json.dump(feature_dis_col_eps,f)

In [215]:
symbol_to_int_map_day = {}
int_to_symbol_map_day = {}
int_to_feature_map_day = {}
count = int(1)

for ep in eps:
    for f,s in zip(feature_col_eps[ep], feature_dis_col_eps[ep]):
        symbol_to_int_map_day[s] = {}
        symbol_to_int_map_day[s]["l"] = count
        symbol_to_int_map_day[s]["m"] = count + 1
        symbol_to_int_map_day[s]["h"] = count + 2
        int_to_symbol_map_day[count] = s + "_l"
        int_to_symbol_map_day[count + 1] = s + "_m"
        int_to_symbol_map_day[count + 2] = s + "_h"
        int_to_feature_map_day[count] = f
        int_to_feature_map_day[count + 1] = f
        int_to_feature_map_day[count + 2] = f

        count += 3
with open(folder_path + "symbol_to_int_map_day.json", "w") as f:
    json.dump(symbol_to_int_map_day, f)
with open(folder_path + "int_to_symbol_map_day.json", "w") as f:
    json.dump(int_to_symbol_map_day, f)
with open(folder_path + "int_to_feature_map_day.json", "w") as f:
    json.dump(int_to_feature_map_day, f)